# Autoassociative Artificial Neural Network

In [1]:
# ===== # IMPORTS # ===== #
import sys
import os
import keras
import random
import numpy as np
import scipy.io as sio

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn import preprocessing

import matplotlib.pyplot as plt
%matplotlib inline

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Using TensorFlow backend.


In [11]:
# ===== # LOAD DATA # ===== #
print(sio.whosmat('../data/data-starplus-04847-v7.mat'))
raw = sio.loadmat('../data/data-starplus-04847-v7.mat')

[('info', (1, 54), 'struct'), ('data', (54, 1), 'cell'), ('meta', (1, 1), 'struct')]


In [ ]:
# raw['meta'].dtype.names
# raw['meta']['rois']

In [17]:
# raw['meta'][0]['rois'][0]['columns'][0][i]

# meta = meta dictionary
# rois = regions of interest dictionary
# columns = indexes of voxels we want to look at for brain activity
# i = based on the region of interest which is obtained using:

num_rois = 25
rois = ['CALC', 'LIPL', 'LT', 'LTRIA', 'LOPER', 'LIPS', 'LDFPF']

vox_ind = [
    raw['meta'][0]['rois'][0]['columns'][0][j][0] 
        for j in range(num_rois - 1) 
            if raw['meta'][0]['rois'][0]['name'][0][j][0] in rois
]

voxels = []

for i in range(len(vox_ind)):
    for j in range(len(vox_ind[i])):
        voxels.append(vox_ind[i][j])
    
len(voxels)

1275

In [27]:
# Take the list of 7 ROIs and grab the relevant voxel values at those 
# relevant slices

stop = 8
slices = [i for i in range(stop)]

# raw['data'][i][0][j][k]
# i = trial
# j = time interval relative to trial (starts at 0), 1 is 500ms, 2 is 1s, etc.
# k = voxels

num_trials = 54
data = []
labels = []
exclude = [0, 26, 27, 32, 37, 46, 51, 53]  # no data here

for ki, k in enumerate(slices):
    data.append([])
    labels.append([])
    offset_i = 0
    
    for i in range(num_trials):
        if (i in exclude):
            offset_i += 1
            continue
        
        data[ki].append([])
        labels[ki].append([])
        
        offset_j = 0
        
        for ji, j in enumerate(voxels):
            if(len(raw['data'][i][0]) == num_trials + 1):
                data[ki][i - offset_i].append([])
                data[ki][i - offset_i][ji - offset_j] = raw['data'][i][0][k][j-1]
            else:
                offset += 1  # if we didn't add an element
            
            labels[ki][i - offset_i].append(
                1 if raw['info'][0][i]['firstStimulus'][0] == 'P' else 0
            )

# The data is now in the format data[slice][trial] = [roi, voxel, values]
# Classification data is in the labels[slice][trial] = []

In [31]:
print(len(data[0][0]))
print(len(labels[0][0]))


1275
1275


## Machine Learning Classification and Preprocessing

For these experiments, we will give an answer based on the frame we're looking at, and potentially a confident value. 

Data input to the algorithms will be given in the form:

```
data[slice] = [array of trials][array of voxel readings]
labels[slice] = [array of classification]
```

In [40]:
frame = 7
num_training_trials = 36

c = list(zip(data[frame], labels[frame]))

random.shuffle(c)

# print(c)

train_x, train_y = zip(*c[num_training_trials:])
test_x, test_y = zip(*c[:num_training_trials])

classifier = MLPClassifier(
    solver = 'sgd', 
    alpha = 1e-5, 
    hidden_layer_sizes = (20, 6),
    random_state = 1,
    max_iter = 1000
)

classifier.fit(train_x, np.array(train_y).reshape(-1))

ValueError: Found input variables with inconsistent numbers of samples: [10, 12750]